In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [14]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False, accent_classification=True)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [22]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

#x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x_conv)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2)

Epoch 1/800
23s - loss: 0.2436 - actual_accuracy: 0.0167 - val_loss: 0.1983 - val_actual_accuracy: 0.0564
Epoch 2/800
26s - loss: 0.1998 - actual_accuracy: 0.0903 - val_loss: 0.1783 - val_actual_accuracy: 0.1499
Epoch 3/800
31s - loss: 0.1662 - actual_accuracy: 0.3068 - val_loss: 0.1515 - val_actual_accuracy: 0.4375
Epoch 4/800
26s - loss: 0.1447 - actual_accuracy: 0.4218 - val_loss: 0.1328 - val_actual_accuracy: 0.5115
Epoch 5/800
26s - loss: 0.1332 - actual_accuracy: 0.4847 - val_loss: 0.1284 - val_actual_accuracy: 0.5567
Epoch 6/800
26s - loss: 0.1243 - actual_accuracy: 0.5333 - val_loss: 0.1145 - val_actual_accuracy: 0.6173
Epoch 7/800
26s - loss: 0.1171 - actual_accuracy: 0.5703 - val_loss: 0.1074 - val_actual_accuracy: 0.6465
Epoch 8/800
27s - loss: 0.1097 - actual_accuracy: 0.6015 - val_loss: 0.1005 - val_actual_accuracy: 0.6856
Epoch 9/800
26s - loss: 0.1058 - actual_accuracy: 0.6224 - val_loss: 0.1041 - val_actual_accuracy: 0.6664
Epoch 10/800
27s - loss: 0.1014 - actual_accur

25s - loss: 0.0406 - actual_accuracy: 0.8168 - val_loss: 0.0799 - val_actual_accuracy: 0.7899
Epoch 79/800
25s - loss: 0.0391 - actual_accuracy: 0.8218 - val_loss: 0.0789 - val_actual_accuracy: 0.7818
Epoch 80/800
28s - loss: 0.0402 - actual_accuracy: 0.8164 - val_loss: 0.0757 - val_actual_accuracy: 0.7853
Epoch 81/800
24s - loss: 0.0397 - actual_accuracy: 0.8186 - val_loss: 0.0739 - val_actual_accuracy: 0.7964
Epoch 82/800
26s - loss: 0.0392 - actual_accuracy: 0.8165 - val_loss: 0.0895 - val_actual_accuracy: 0.7686
Epoch 83/800
27s - loss: 0.0392 - actual_accuracy: 0.8187 - val_loss: 0.0729 - val_actual_accuracy: 0.7952
Epoch 84/800
25s - loss: 0.0390 - actual_accuracy: 0.8193 - val_loss: 0.0826 - val_actual_accuracy: 0.7768
Epoch 85/800
25s - loss: 0.0385 - actual_accuracy: 0.8170 - val_loss: 0.0817 - val_actual_accuracy: 0.7860
Epoch 86/800
26s - loss: 0.0401 - actual_accuracy: 0.8141 - val_loss: 0.0857 - val_actual_accuracy: 0.7864
Epoch 87/800
26s - loss: 0.0396 - actual_accuracy:

24s - loss: 0.0320 - actual_accuracy: 0.8324 - val_loss: 0.0822 - val_actual_accuracy: 0.7818
Epoch 156/800
24s - loss: 0.0305 - actual_accuracy: 0.8385 - val_loss: 0.1007 - val_actual_accuracy: 0.7753
Epoch 157/800
25s - loss: 0.0314 - actual_accuracy: 0.8336 - val_loss: 0.0963 - val_actual_accuracy: 0.7780
Epoch 158/800
24s - loss: 0.0316 - actual_accuracy: 0.8348 - val_loss: 0.0949 - val_actual_accuracy: 0.7860
Epoch 159/800
24s - loss: 0.0315 - actual_accuracy: 0.8345 - val_loss: 0.0971 - val_actual_accuracy: 0.7699
Epoch 160/800
24s - loss: 0.0306 - actual_accuracy: 0.8409 - val_loss: 0.0974 - val_actual_accuracy: 0.7834
Epoch 161/800
24s - loss: 0.0313 - actual_accuracy: 0.8349 - val_loss: 0.0873 - val_actual_accuracy: 0.7726
Epoch 162/800
24s - loss: 0.0321 - actual_accuracy: 0.8323 - val_loss: 0.0935 - val_actual_accuracy: 0.7795
Epoch 163/800
25s - loss: 0.0315 - actual_accuracy: 0.8312 - val_loss: 0.1022 - val_actual_accuracy: 0.7671
Epoch 164/800
25s - loss: 0.0312 - actual_

24s - loss: 0.0286 - actual_accuracy: 0.8426 - val_loss: 0.1032 - val_actual_accuracy: 0.7765
Epoch 232/800
24s - loss: 0.0291 - actual_accuracy: 0.8347 - val_loss: 0.0969 - val_actual_accuracy: 0.7684
Epoch 233/800
24s - loss: 0.0294 - actual_accuracy: 0.8364 - val_loss: 0.0885 - val_actual_accuracy: 0.8002
Epoch 234/800
24s - loss: 0.0292 - actual_accuracy: 0.8346 - val_loss: 0.0985 - val_actual_accuracy: 0.7837
Epoch 235/800
24s - loss: 0.0286 - actual_accuracy: 0.8374 - val_loss: 0.1144 - val_actual_accuracy: 0.7822
Epoch 236/800
24s - loss: 0.0290 - actual_accuracy: 0.8359 - val_loss: 0.0951 - val_actual_accuracy: 0.7853
Epoch 237/800
25s - loss: 0.0295 - actual_accuracy: 0.8355 - val_loss: 0.1122 - val_actual_accuracy: 0.7857
Epoch 238/800
24s - loss: 0.0297 - actual_accuracy: 0.8295 - val_loss: 0.1111 - val_actual_accuracy: 0.7795
Epoch 239/800
24s - loss: 0.0288 - actual_accuracy: 0.8375 - val_loss: 0.1046 - val_actual_accuracy: 0.7791
Epoch 240/800
24s - loss: 0.0287 - actual_

23s - loss: 0.0266 - actual_accuracy: 0.8445 - val_loss: 0.1067 - val_actual_accuracy: 0.7857
Epoch 308/800
24s - loss: 0.0276 - actual_accuracy: 0.8412 - val_loss: 0.1010 - val_actual_accuracy: 0.7895
Epoch 309/800
24s - loss: 0.0276 - actual_accuracy: 0.8390 - val_loss: 0.1144 - val_actual_accuracy: 0.7745
Epoch 310/800
24s - loss: 0.0276 - actual_accuracy: 0.8403 - val_loss: 0.1135 - val_actual_accuracy: 0.7899
Epoch 311/800
23s - loss: 0.0269 - actual_accuracy: 0.8477 - val_loss: 0.1090 - val_actual_accuracy: 0.7853
Epoch 312/800
23s - loss: 0.0278 - actual_accuracy: 0.8423 - val_loss: 0.1003 - val_actual_accuracy: 0.7845
Epoch 313/800
23s - loss: 0.0271 - actual_accuracy: 0.8437 - val_loss: 0.1080 - val_actual_accuracy: 0.7918
Epoch 314/800
24s - loss: 0.0292 - actual_accuracy: 0.8343 - val_loss: 0.0985 - val_actual_accuracy: 0.8044
Epoch 315/800
24s - loss: 0.0267 - actual_accuracy: 0.8438 - val_loss: 0.1164 - val_actual_accuracy: 0.7757
Epoch 316/800
23s - loss: 0.0267 - actual_

...

24s - loss: 0.0260 - actual_accuracy: 0.8431 - val_loss: 0.1228 - val_actual_accuracy: 0.7738
Epoch 460/800
24s - loss: 0.0259 - actual_accuracy: 0.8438 - val_loss: 0.1222 - val_actual_accuracy: 0.7795
Epoch 461/800
24s - loss: 0.0270 - actual_accuracy: 0.8374 - val_loss: 0.1179 - val_actual_accuracy: 0.7803
Epoch 462/800
23s - loss: 0.0259 - actual_accuracy: 0.8494 - val_loss: 0.1363 - val_actual_accuracy: 0.7845
Epoch 463/800
22s - loss: 0.0259 - actual_accuracy: 0.8450 - val_loss: 0.1366 - val_actual_accuracy: 0.7826
Epoch 464/800
24s - loss: 0.0265 - actual_accuracy: 0.8427 - val_loss: 0.1156 - val_actual_accuracy: 0.7814
Epoch 465/800
22s - loss: 0.0266 - actual_accuracy: 0.8415 - val_loss: 0.1204 - val_actual_accuracy: 0.7880
Epoch 466/800
23s - loss: 0.0265 - actual_accuracy: 0.8417 - val_loss: 0.1138 - val_actual_accuracy: 0.7926
Epoch 467/800
27s - loss: 0.0262 - actual_accuracy: 0.8434 - val_loss: 0.1121 - val_actual_accuracy: 0.7836
Epoch 468/800
24s - loss: 0.0266 - actual_

24s - loss: 0.0261 - actual_accuracy: 0.8445 - val_loss: 0.1304 - val_actual_accuracy: 0.7883
Epoch 536/800
24s - loss: 0.0261 - actual_accuracy: 0.8465 - val_loss: 0.1120 - val_actual_accuracy: 0.7818
Epoch 537/800
23s - loss: 0.0262 - actual_accuracy: 0.8406 - val_loss: 0.1149 - val_actual_accuracy: 0.7761
Epoch 538/800
23s - loss: 0.0265 - actual_accuracy: 0.8397 - val_loss: 0.1305 - val_actual_accuracy: 0.7676
Epoch 539/800
23s - loss: 0.0263 - actual_accuracy: 0.8424 - val_loss: 0.1396 - val_actual_accuracy: 0.7696
Epoch 540/800
23s - loss: 0.0265 - actual_accuracy: 0.8431 - val_loss: 0.1284 - val_actual_accuracy: 0.7864
Epoch 541/800
24s - loss: 0.0260 - actual_accuracy: 0.8458 - val_loss: 0.1182 - val_actual_accuracy: 0.7765
Epoch 542/800
24s - loss: 0.0262 - actual_accuracy: 0.8490 - val_loss: 0.1375 - val_actual_accuracy: 0.7600
Epoch 543/800
25s - loss: 0.0267 - actual_accuracy: 0.8452 - val_loss: 0.1456 - val_actual_accuracy: 0.7619
Epoch 544/800
24s - loss: 0.0264 - actual_

24s - loss: 0.0254 - actual_accuracy: 0.8483 - val_loss: 0.1373 - val_actual_accuracy: 0.7692
Epoch 612/800
24s - loss: 0.0269 - actual_accuracy: 0.8408 - val_loss: 0.1259 - val_actual_accuracy: 0.7665
Epoch 613/800
24s - loss: 0.0252 - actual_accuracy: 0.8472 - val_loss: 0.1351 - val_actual_accuracy: 0.7768
Epoch 614/800
24s - loss: 0.0256 - actual_accuracy: 0.8467 - val_loss: 0.1423 - val_actual_accuracy: 0.7803
Epoch 615/800
25s - loss: 0.0260 - actual_accuracy: 0.8469 - val_loss: 0.1222 - val_actual_accuracy: 0.7849
Epoch 616/800
24s - loss: 0.0261 - actual_accuracy: 0.8425 - val_loss: 0.1269 - val_actual_accuracy: 0.7765
Epoch 617/800
24s - loss: 0.0259 - actual_accuracy: 0.8451 - val_loss: 0.1224 - val_actual_accuracy: 0.7699
Epoch 618/800
24s - loss: 0.0261 - actual_accuracy: 0.8453 - val_loss: 0.1258 - val_actual_accuracy: 0.7692
Epoch 619/800
25s - loss: 0.0254 - actual_accuracy: 0.8481 - val_loss: 0.1256 - val_actual_accuracy: 0.7703
Epoch 620/800
24s - loss: 0.0263 - actual_

24s - loss: 0.0265 - actual_accuracy: 0.8431 - val_loss: 0.1356 - val_actual_accuracy: 0.7857
Epoch 688/800
24s - loss: 0.0265 - actual_accuracy: 0.8431 - val_loss: 0.1439 - val_actual_accuracy: 0.7849
Epoch 689/800
24s - loss: 0.0264 - actual_accuracy: 0.8423 - val_loss: 0.1178 - val_actual_accuracy: 0.7661
Epoch 690/800
23s - loss: 0.0260 - actual_accuracy: 0.8447 - val_loss: 0.1137 - val_actual_accuracy: 0.7705
Epoch 691/800
24s - loss: 0.0265 - actual_accuracy: 0.8397 - val_loss: 0.1354 - val_actual_accuracy: 0.7837
Epoch 692/800
24s - loss: 0.0267 - actual_accuracy: 0.8423 - val_loss: 0.1322 - val_actual_accuracy: 0.7722
Epoch 693/800
24s - loss: 0.0254 - actual_accuracy: 0.8481 - val_loss: 0.1326 - val_actual_accuracy: 0.7738
Epoch 694/800
24s - loss: 0.0267 - actual_accuracy: 0.8418 - val_loss: 0.1298 - val_actual_accuracy: 0.7726
Epoch 695/800
24s - loss: 0.0257 - actual_accuracy: 0.8481 - val_loss: 0.1502 - val_actual_accuracy: 0.7837
Epoch 696/800
24s - loss: 0.0255 - actual_

25s - loss: 0.0261 - actual_accuracy: 0.8470 - val_loss: 0.1455 - val_actual_accuracy: 0.7841
Epoch 764/800
24s - loss: 0.0259 - actual_accuracy: 0.8476 - val_loss: 0.1342 - val_actual_accuracy: 0.7734
Epoch 765/800
25s - loss: 0.0253 - actual_accuracy: 0.8543 - val_loss: 0.1433 - val_actual_accuracy: 0.7669
Epoch 766/800
25s - loss: 0.0258 - actual_accuracy: 0.8472 - val_loss: 0.1355 - val_actual_accuracy: 0.7876
Epoch 767/800
25s - loss: 0.0267 - actual_accuracy: 0.8465 - val_loss: 0.1655 - val_actual_accuracy: 0.7584
Epoch 768/800
25s - loss: 0.0264 - actual_accuracy: 0.8448 - val_loss: 0.1569 - val_actual_accuracy: 0.7711
Epoch 769/800
25s - loss: 0.0255 - actual_accuracy: 0.8524 - val_loss: 0.1369 - val_actual_accuracy: 0.7803
Epoch 770/800
25s - loss: 0.0257 - actual_accuracy: 0.8473 - val_loss: 0.1410 - val_actual_accuracy: 0.7627
Epoch 771/800
25s - loss: 0.0260 - actual_accuracy: 0.8482 - val_loss: 0.1245 - val_actual_accuracy: 0.7845
Epoch 772/800
25s - loss: 0.0265 - actual_

In [20]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()